In [1]:
#IMPORTAÇÃO DOS MODULOS E CHAMADA DAS TABELAS
import os
import pandas as pd
import glob
import pyodbc

In [2]:
#FUNÇAO PARA CONEXAO COM O BANCO DE DADOS SQL
def executar_conexao():
    server = 'TBDBL-GLEISSON'
    database = 'Dados_Pavimento'
    username = 'gleisson.santos'
    password = '12345'
    conexao = pyodbc.connect(
        'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
    print('Conexão ao BD realizada com Socesso!')
    return conexao
executar_conexao()

Conexão ao BD realizada com Socesso!


In [4]:
path = r'C:\Users\gleisson.santos\Desktop\Projetos Checkd Git\# 1 Extração e Automação de dados\Extração da dados\Extração de dados Selenium e BD\Extracao-de-dados-Selenium-BD\aa'
files_xls = os.listdir(path)

df = pd.DataFrame()
for f in files_xls:
    file_path = os.path.join(path, f)
    data = pd.read_html(file_path)
    data = data[1]
    data = data.dropna(axis=1, how='all')
    df = df.append(data)   
df



,SS,OS,Tipo,Especificação,Serviço,Unid Atual (OS),Equipe,Origem da SS,Localidade,Bairro,...,Encerramento,Mot de Encerr da OS,Sit da OS,Insumos,Func. Encerrou OS,SS Atendida,Tipo de Operação,Data/Hora Última Tramitação da OS,Ini do Serviço,Fim do Serviço
0,979532274,3,26 - MANUTENCAO DE HIDROMETRO,19 - REVISAO HIDROMETRO PARADO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,127-A - GENILSON LIMA VITORIA,Interna,700 - LAURO DE FREITAS,4 - AREIA BRANCA,...,27/02/2023 12:12:08,343 - HIDROMETRO INTERNO,Concluída Não Executada,0,94430 - EMERSON DA SILVA SALES,Sim,Comercial,31/01/2023 13:14:00,NaN,NaN
1,980733964,2,26 - MANUTENCAO DE HIDROMETRO,19 - REVISAO HIDROMETRO PARADO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,127-A - GENILSON LIMA VITORIA,Interna,700 - LAURO DE FREITAS,19 - JAMBEIRO,...,27/02/2023 12:20:49,102 - LIGACAO ENCONTRADA CORTADA,Concluída Não Executada,0,94430 - EMERSON DA SILVA SALES,Sim,Comercial,13/02/2023 16:26:53,NaN,NaN
2,979985343,3,26 - MANUTENCAO DE HIDROMETRO,4 - SUBSTITUICAO DE HIDROMETRO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,104-A - ALBERICO MAGNO BRITO FILHO,Interna,700 - LAURO DE FREITAS,36 - CAIXA DAGUA,...,27/02/2023 12:23:44,8 - NAO PERMITIDO PELO USUARIO,Concluída Não Executada,0,94430 - EMERSON DA SILVA SALES,Não,Comercial,16/02/2023 13:46:00,NaN,NaN
3,980905823,2,26 - MANUTENCAO DE HIDROMETRO,19 - REVISAO HIDROMETRO PARADO,12 - SUBSTITUICAO HID.,88880068 - CONSORCIO NOVA BOLANDEIRA,111-A - VANILTON DA SILVA DE JESUS,Interna,700 - LAURO DE FREITAS,23 - ITINGA SSA,...,26/02/2023 13:49:41,8 - NAO PERMITIDO PELO USUARIO,Concluída Não Executada,0,94430 - EMERSON DA SILVA SALES,Sim,Comercial,23/02/2023 08:35:53,NaN,NaN
4,980609447,3,26 - MANUTENCAO DE HIDROMETRO,19 - REVISAO HIDROMETRO PARADO,37 - VISITA,88880068 - CONSORCIO NOVA BOLANDEIRA,111-A - VANILTON DA SILVA DE JESUS,Interna,700 - LAURO DE FREITAS,3 - CAJI,...,28/02/2023 14:26:01,7 - INVIABILIDADE TECNICA,Concluída Não Executada,0,203862 - DAIANA PEREIRA SANTOS,Sim,Comercial,22/02/2023 08:35:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,981796063,1,43 - VAZAMENTO DE AGUA,3 - VAZAMENTO NO HIDROMETRO,290 - VAZAMENTO NO HIDROMETRO,88880068 - CONSORCIO NOVA BOLANDEIRA,228-A - IVAN HONORATO DOS SANTOS,Cliente,900 - SALVADOR UMB,53 - ITAIGARA,...,12/03/2023 19:29:22,0 - NAO INFORMADO,Concluída Executada,5,900032 - INTEGRACAO DIGITEAM,Sim,Água,12/03/2023 16:20:46,12/03/2023 18:19:00,12/03/2023 19:29:00
638,981796164,1,45 - VERIFICACAO DE FALTA DE AGUA,1 - FALTA DE AGUA NO IMOVEL,364 - VERIF FALTA AGUA IMOVEL,88880068 - CONSORCIO NOVA BOLANDEIRA,152-A - ALCIDES P WOYTSIAK P FILHO,Cliente,900 - SALVADOR UMB,16 - BOCA DO RIO,...,12/03/2023 17:52:26,0 - NAO INFORMADO,Concluída Executada,0,201899 - APARECIDA LEAL GONCALVES,Sim,Água,12/03/2023 16:20:46,12/03/2023 16:40:00,12/03/2023 16:50:00
639,981796527,1,45 - VERIFICACAO DE FALTA DE AGUA,1 - FALTA DE AGUA NO IMOVEL,364 - VERIF FALTA AGUA IMOVEL,88880068 - CONSORCIO NOVA BOLANDEIRA,143-A - YAN QUARESMA DOS SANTOS,Cliente,900 - SALVADOR UMB,15 - PITUBA,...,12/03/2023 18:20:11,0 - NAO INFORMADO,Concluída Executada,0,900032 - INTEGRACAO DIGITEAM,Sim,Água,12/03/2023 17:16:25,12/03/2023 18:11:00,12/03/2023 18:19:00
640,981796734,1,25 - MANUTENCAO DE ESGOTO,45 - CONSERTO DE REDE DE ESGOTO,87 - CONS REDE COLET ESG C/PAV,88880068 - CONSORCIO NOVA BOLANDEIRA,430E - CHARLES MARINHO DOS SANTOS,Interna,900 - SALVADOR UMB,15 - PITUBA,...,12/03/2023 22:41:58,0 - NAO INFORMADO,Concluída Executada,1,900032 - INTEGRACAO DIGITEAM,Sim,Esgoto,12/03/2023 17:41:21,12/03/2023 18:38:00,12/03/2023 22:40:00


In [5]:
df.to_excel('Dados.xlsx', index=False)

In [204]:
#INSERT DA TABALA TRATADA AO BANCO DE DADOS SQL
cursor = executar_conexao().cursor()
for index, row in df.iterrows():
    cursor.execute("INSERT INTO Material (SS, OS, Serviço, Equipe, Localidade, Bairro, Logradouro, Encerramento, ObsOS, ObsEnc) values(?,?,?,?,?,?,?,?,?,?)",
                row.SS, row.OS, row.Serviço, row.Equipe, row.Localidade,  row.Bairro, row.Logradouro, row.Encerramento, row.ObsOS, row.ObsEnc)

cursor.commit()
cursor.close()

Conexão Ok


ProgrammingError: ('42000', '[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]O fluxo de protocolo da chamada de procedimento remoto (RPC) do protocolo TDS recebido está incorreto. Parâmetro 13 (""): o valor fornecido não é uma instância válida do tipo de dados float. Procure valores inválidos nos dados de origem. Dados do tipo numérico com escala maior que precisão são um exemplo de um valor inválido. (8023) (SQLExecDirectW)')

In [171]:
#APAGAR REGISTROS NA TABELA
cursor = executar_conexao().cursor()
cursor.execute('TRUNCATE TABLE Material')
cursor.commit()

Conexão Ok
